In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
from PIL import Image
import os

In [12]:
def convert_to_rgba(image_path):
    image = Image.open(image_path)
    if image.mode != 'RGBA':
        image = image.convert('RGBA')
        image.save(image_path)  # Overwrite the original image with the converted RGBA image

# Example usage
image_directory = 'Logo'
for filename in os.listdir(image_directory):
    if filename.endswith('.png'):
        convert_to_rgba(os.path.join(image_directory, filename))

In [14]:
# Define CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 1 output node for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'],
              run_eagerly=True)


# Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
        'Logo',
        target_size=(224, 224),
        batch_size=15,
        class_mode='binary')  # binary labels: 'real' and 'fake'

# Train the model
model.fit(
    train_generator,
    epochs=10,
    steps_per_epoch=train_generator.samples // train_generator.batch_size
)


Found 25 images belonging to 2 classes.
Epoch 1/10
1/1 [==============================] - 14s 14s/step - loss: 0.6934 - accuracy: 0.6667
Epoch 2/10
1/1 [==============================] - 4s 4s/step - loss: 1.5452 - accuracy: 0.6000
Epoch 3/10
1/1 [==============================] - 5s 5s/step - loss: 0.6637 - accuracy: 0.7333
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 0.6395 - accuracy: 0.5333
Epoch 5/10
1/1 [==============================] - 7s 7s/step - loss: 0.5473 - accuracy: 0.8000
Epoch 6/10
1/1 [==============================] - 5s 5s/step - loss: 0.5111 - accuracy: 0.8000
Epoch 7/10
1/1 [==============================] - 7s 7s/step - loss: 0.5135 - accuracy: 0.6667
Epoch 8/10
1/1 [==============================] - 6s 6s/step - loss: 0.5972 - accuracy: 0.6000
Epoch 9/10
1/1 [==============================] - 4s 4s/step - loss: 0.5294 - accuracy: 0.7000
Epoch 10/10
1/1 [==============================] - 5s 5s/step - loss: 0.3324 - accuracy: 1.0000


In [15]:
# Load and augment validation data
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
        'Logo',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

# Evaluate the model on validation data
evaluation = model.evaluate(validation_generator)

print("Validation Accuracy:", evaluation[1])


Found 25 images belonging to 2 classes.


1/1 [==============================] - 2s 2s/step - loss: 0.3940 - accuracy: 0.8800
Validation Accuracy: 0.8799999952316284


In [10]:
# Save the trained model
model.save('logo_classifier_model.h5')

In [16]:
import pickle

In [17]:
# Save the trained model
with open('logo_classifier_model.pkl', 'wb') as f:
    pickle.dump(model, f)